In [1]:
import pandas as pd
import sys
import os
sys.path.append("../")
sys.path.append("../..")

# Data Ingestion Initialization

## DB Client 

In [2]:
from KETIPreDataIngestion.KETI_setting import influx_setting_KETI as ins
from KETIPreDataIngestion.data_influx import influx_Client

DBClient = influx_Client.influxClient(ins)
db_list = DBClient.get_DBList()

In [3]:
db_list

['energy_solar',
 'traffic_seoul_subway',
 'traffic_seoul_bus',
 'test',
 'energy_electronic_power',
 'test_bio_covid',
 'test_indoor_high',
 'air_indoor_중학교',
 'traffic_seoul_bike',
 'traffic_public_parking',
 'energy_electricity_transaction',
 'energy_fire_power',
 'traffic_expressway',
 'farm_inner_air',
 'farm_outdoor_weather',
 'farm_outdoor_weather_clean',
 'farm_outdoor_air',
 'farm_outdoor_air_clean',
 'air_outdoor_에어코리아',
 'energy_water_discharge',
 'energy_wind_power',
 'city_exhibition_entrance_status',
 'bio_covid_vaccinations',
 'air_indoor_유치원',
 'air_indoor_요양원',
 'air_indoor_아파트',
 'air_indoor_도서관',
 'air_indoor_경로당',
 'air_indoor_초등학교',
 'air_indoor_체육시설',
 'air_indoor_어린이집',
 'air_indoor_고등학교',
 'air_outdoor',
 'bio_covid_infected_world',
 'bio_covid_infected_korea',
 'bio_test_infected',
 'keti_rtd',
 'culture_subway',
 'farm_strawberry_knu',
 'weather_korean_nws',
 'finance_korean_stock',
 'farm_strawberry_gunwi',
 'farm_strawberry_awon',
 'farm_strawberry_jinan']

## DB and Measurement name setting

In [34]:
db_name  = 'finance_korean_stock'

In [35]:
ms_list = DBClient.measurement_list(db_name)
ms_name = ms_list[0]

# Data Ingestion Methods

## pure data ingestion

### Ingest Full Data

In [36]:
print(db_name, ms_name)

finance_korean_stock finance


In [37]:
feature_list = DBClient.get_fieldList(db_name, ms_name)
number = "100"
data = DBClient.get_datafront_by_num(number, db_name, ms_name)
data.head()

,adj_close_value,close_value,company,country,exchange,high_value,industry,low_value,open_value,ticker,volume_value
time,,,,,,,,,,,
2021-04-01 00:00:00+00:00,633.598572,637.099976,ASML 홀딩 NV ADR,US,NASDAQ,639.48999,반도체 및 반도체 장비,619.750000,619.799988,ASML,1908300.0
2021-04-05 00:00:00+00:00,636.969971,640.489990,ASML 홀딩 NV ADR,US,NASDAQ,653.00000,반도체 및 반도체 장비,638.030029,651.650024,ASML,1529800.0


In [38]:
first = DBClient.get_first_time(db_name, ms_name)
last = DBClient.get_last_time(db_name, ms_name)
print(first, last)

                      adj_close_value  close_value         company country  \
time                                                                         
2021-04-01T00:00:00Z       633.598572   637.099976  ASML 홀딩 NV ADR      US   

                     exchange  high_value      industry  low_value  \
time                                                                 
2021-04-01T00:00:00Z   NASDAQ   639.48999  반도체 및 반도체 장비     619.75   

                      open_value ticker  volume_value  
time                                                   
2021-04-01T00:00:00Z  619.799988   ASML     1908300.0  
                      adj_close_value  close_value company country exchange  \
time                                                                          
2021-04-29T00:00:00Z        97.055939    99.089996    헤즈브로      US   NASDAQ   

                      high_value industry  low_value  open_value ticker  \
time                                                                     

In [39]:
full_data = DBClient.get_data(db_name, ms_name)

In [40]:
query_end_time = last
query_start_time = first
bind_params = {'end_time':query_end_time, 'start_time': query_start_time}
time_data = DBClient.get_data_by_time(bind_params, db_name, ms_name)
print(time_data.head())
print(time_data.tail())

                           adj_close_value  close_value             company  \
time                                                                          
2021-04-01 00:00:00+00:00       633.598572   637.099976      ASML 홀딩 NV ADR   
2021-04-05 00:00:00+00:00       636.969971   640.489990      ASML 홀딩 NV ADR   
2021-04-06 00:00:00+00:00        54.364899    54.840000                컴캐스트   
2021-04-07 00:00:00+00:00       485.309998   485.309998  지브라 테크놀로지스 Class A   
2021-04-08 00:00:00+00:00        51.719639    52.009998  뉴 포트리스 에너지 Class A   

                          country exchange  high_value      industry  \
time                                                                   
2021-04-01 00:00:00+00:00      US   NASDAQ  639.489990  반도체 및 반도체 장비   
2021-04-05 00:00:00+00:00      US   NASDAQ  653.000000  반도체 및 반도체 장비   
2021-04-06 00:00:00+00:00      US   NASDAQ   55.049999      미디어 및 출판   
2021-04-07 00:00:00+00:00      US   NASDAQ  493.440002    전자 장비 및 부품   
2021-04-08 00:

In [41]:
number="20"
time_data = DBClient.get_datafront_by_num(number, db_name, ms_name)
print(time_data.head())
print(time_data.tail())

                           adj_close_value  close_value         company  \
time                                                                      
2021-04-01 00:00:00+00:00       633.598572   637.099976  ASML 홀딩 NV ADR   

                          country exchange  high_value      industry  \
time                                                                   
2021-04-01 00:00:00+00:00      US   NASDAQ   639.48999  반도체 및 반도체 장비   

                           low_value  open_value ticker  volume_value  
time                                                                   
2021-04-01 00:00:00+00:00     619.75  619.799988   ASML     1908300.0  
                           adj_close_value  close_value         company  \
time                                                                      
2021-04-01 00:00:00+00:00       633.598572   637.099976  ASML 홀딩 NV ADR   

                          country exchange  high_value      industry  \
time                                       

In [42]:
days = 7 
bind_params = {'end_time': query_end_time,"days":str(days)+"d"}
day_data = DBClient.get_data_by_days(bind_params, db_name, ms_name)
print(day_data.head(), day_data.tail())

                           adj_close_value  close_value             company  \
time                                                                          
2021-04-22 00:00:00+00:00       643.533752   647.090027      ASML 홀딩 NV ADR   
2021-04-23 00:00:00+00:00        53.918793    54.389999                컴캐스트   
2021-04-26 00:00:00+00:00       666.755432   670.440002      ASML 홀딩 NV ADR   
2021-04-27 00:00:00+00:00        53.829575    54.299999                컴캐스트   
2021-04-28 00:00:00+00:00       496.600006   496.600006  지브라 테크놀로지스 Class A   

                          country exchange  high_value      industry  \
time                                                                   
2021-04-22 00:00:00+00:00      US   NASDAQ  665.869995  반도체 및 반도체 장비   
2021-04-23 00:00:00+00:00      US   NASDAQ   54.709999      미디어 및 출판   
2021-04-26 00:00:00+00:00      US   NASDAQ  675.650024  반도체 및 반도체 장비   
2021-04-27 00:00:00+00:00      US   NASDAQ   54.770000      미디어 및 출판   
2021-04-28 00:

In [43]:
day_data.['day_data']

SyntaxError: invalid syntax (<ipython-input-43-159ce2604080>, line 1)

In [44]:
tag_list = DBClient.get_fieldList(db_name, ms_name)

In [45]:
tag_list

['adj_close_value',
 'close_value',
 'high_value',
 'low_value',
 'open_value',
 'volume_value']

In [46]:
tag_key = 'company'
tag_value='컴캐스트'

df = DBClient.get_TagGroupData(db_name, ms_name, tag_key, tag_value)

select * from "finance" WHERE "company" = "컴캐스트"


In [29]:
df

""
